In [1]:
!pip install -q tf-nightly

     |████████████████████████████████| 517.1MB 29kB/s 
     |████████████████████████████████| 2.8MB 50.6MB/s 
     |████████████████████████████████| 460kB 47.6MB/s 


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os


In [0]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

In [4]:
for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
parent_dir = os.path.dirname(text_dir)
parent_dir

811008/807992 [==============================] - 0s 0us/step


'/root/.keras/datasets'

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

labeled_data_sets = []
for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000


In [0]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False
)

In [10]:
for ex in all_labeled_data.take(5):
  print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'My brave, my beautiful, of heroes chief!'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'ill has now been spoken may the gods bring it to nothing."'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'sacrifices of bulls and rams. These were commanded by Menestheus, son'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Urge me no longer, at a time like this,'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Wielded with dire intent; the brazen gleam'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [23]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

17178

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [14]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'My brave, my beautiful, of heroes chief!'


In [15]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[9865, 10333, 9169, 5857, 8909, 21, 1467]


In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label 
  

In [0]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))


In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [21]:
sample_text, sample_labels = next(iter(test_data))
sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=
 array([ 9865, 10333,  9169,  5857,  8909,    21,  1467,     0,     0,
            0,     0,     0,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [0]:
vocab_size +=1

In [0]:
model = tf.keras.Sequential()

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

In [0]:
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

model.add(tf.keras.layers.Dense(3))


In [0]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [32]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
697/697 [==============================] - 27s 39ms/step - loss: 0.5678 - accuracy: 0.6995 - val_loss: 0.4083 - val_accuracy: 0.8160
Epoch 2/3
697/697 [==============================] - 26s 38ms/step - loss: 0.3177 - accuracy: 0.8600 - val_loss: 0.3952 - val_accuracy: 0.8294
Epoch 3/3
697/697 [==============================] - 27s 38ms/step - loss: 0.2309 - accuracy: 0.9008 - val_loss: 0.3817 - val_accuracy: 0.8338


In [34]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}%'.format(eval_loss, 100*eval_acc))

79/79 [==============================] - 2s 26ms/step - loss: 0.3817 - accuracy: 0.8338

Eval loss: 0.382, Eval accuracy: 83.380%


In [37]:
!pwd

/content


In [40]:
cd drive/My Drive

/content/drive/My Drive


In [41]:
!pwd

/content/drive/My Drive


In [42]:
cd Tensorflow/

/content/drive/My Drive/Tensorflow


In [44]:
!git init

Reinitialized existing Git repository in /content/drive/My Drive/Tensorflow/.git/


In [79]:
!git remote add origin https://github.com/phu-bui/Tensorflow.git

fatal: remote origin already exists.


In [0]:
!git remote rm origin

In [82]:
!git push -u origin master

fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [66]:
!git config --global user.email "phu-bui@git.com"

phu-bui@git.com


In [0]:
!git config --global user.name "phu-bui"
